# Opciones

Una opción es un contrato que da al comprador, el derecho de entrar en otro contrato durante un periodo de tiempo en particular. Este contrato recibe el nombre de **derivado** ya que su valor depende del valor de otro contrato, el **subyacente**. Este contrato subyacente puede ser una acción, un bono o algún activo financiero como un commodity, una moneda o una tasa de interés.

Si la opción da el derecho de comprar el subyacente, se dice que es una **opción call**, en cambio, si se da el derecho de venderlo, se habla de una **opción put**.

En este notebook consideraremos únicamente aquellas opciones que sólo tienen una fecha de ejercicio, es decir, **opciones europeas**.

## Fórmula de Black-Scholes-Merton
Para valuar el precio de una opción call europea (con un payoff de $max(S_{T} - K, 0)$ ) con subyacente $S$ y precio de ejercico (strike) $K$, el modelo de Black-Scholes-Merton establace la siguiente fórmula:

$$
\begin{array}{rll}
 Call(S_{t}, K, t, T, r, \sigma) & = & S_{t}N(d_1) - e^{-r(T-t)} K N(d_2) \\
 N(d) & = &\dfrac{1}{\sqrt{2\pi}} \int_{- \infty}^{d} e^{- 0.5 x^2}dx \\
d_1 & = & \dfrac{ Ln \Big(\dfrac{S_t}{K}\Big) + \Big( r + \dfrac{\sigma^2}{2} \Big) \Big(T - t \Big) }{ \sigma \sqrt{T-t} } \\
d_2 & = & \dfrac{ Ln \Big(\dfrac{S_t}{K}\Big) - \Big( r + \dfrac{\sigma^2}{2} \Big) \Big(T - t \Big) }{ \sigma \sqrt{T-t} } 
\end{array}
$$

en donde $r$ es la tasa (anual) libre de riesgo, $\sigma$ es la volatilidad anual del subyacente, $T$ es la fecha de vencimiento de la opción, $t$ es la fecha de valuación y $N$ es la función de distribución de una variable aleatoria normal estándar. La diferencia $T-t$ se expresa en fracciones de año.

### Ejercicio

+ Implemente la ecuación de Black-Scholes-Merton para valuar una opción call europea y prueba su función con los siguientes datos:
    * $S_{t} = 100$
    * $K = 105$
    * $\sigma = 0.2$
    * $r = 0.05$
    * $T = 2020/08/18$
    * $t = 2019/08/18$
    * Considere un año de 365 días.

**Sugerencia** Para calcular la función $N(d)$, puede utilizar el método *cdf* de la clase norm
```python
from scipy.stats import norm
help(norm.cdf)
```
    

In [1]:
#Escriba aquí su código
import numpy as np
from scipy.stats import norm

## Valuación utilizando simulación Monte Carlo
La simulación Monte Carlo es uno de los métodos más importantes en finanzas. Particularmente, es de gran ayuda al momento de valuar instrumentos financieros derivados.

En el modelo de Black-Scholes-Merton (bajo la medida neutral al riesgo) los precios siguen un movimiento browniano geométrico que satisface la siguiente ecuación diferencial estocástica:

$$
dS_{t} = rS_{t}dt + \sigma S_{t} dW_{t}
$$
en donde $\{W_t\}$ es un proceso de Wiener (movimiento browniano).

Para poder valuar un derivado utilizando simulación Monte Carlo, es necesario primero simular los precios del subyacente de acuerdo a la ecuación que los describe. Utilizando el esquema de discretización de Euler, podemos simular $S_t$ de la siguiente manera:

$$
S_{t} = S_{t - \Delta t} exp \Big \{ \Big ( r - 0.5 \sigma^2 \Big)\Delta t + \sigma \sqrt{\Delta t} z_{t} \Big \}.
$$

en donde $z_t$ es una variable aleatoria normal estándar y $0 < \Delta t < T$ un intervalo de tiempo (pequeño).

Los pasos para valuar un instrumento derivado con payoff $H$ a través del método de Monte Carlo, son los siguientes:

1. Dividir el intervalo $[t, T]$ en $N$ subintervalos del mismo tamaño $\Delta t$, es decir, $\Delta t = \dfrac{T-t}{N}$ (recuerde que $T-t$ se expresa como fracción de año.

2. Para cada simulación $i = 1, \ldots, M$:

    a. Para cada tiempo $t \in \{\Delta t, 2 \Delta t, \ldots, T\}$ simula $S_{t}$ (esta es una trayectoria del precio desde $S_{t}$ hasta $S_{T}$).
    
    b. Con los datos de la trayectoria simulada, se obtiene el valor del payoff $H$, en el caso de un call europeo el payoff es $H(S) = max(S_{T} - K, 0)$.
    
3. El valor del derivado es el promedio los payoff obtenidos en cada trayectoría, descontándolo utilizando la tasa libre de riesgo, es decir:

$$\text{Valor d = e^{-r \big( T - t \big)} \dfrac{1}{M} \sum_{i = 1}^{M} H_{i}(S).$$

### Ejercicio

+ Utilizando los datos del ejercicio anterior, obtenga el precio de la call europea utilizando simulación Monte Carlo.

**Sugerencia**
Para obtener una realización de la variable $z_t$ puede utilizar la función *normal* del módulo *random* de numpy
```python
import numpy as np
help(np.random.normal)
```

In [2]:
#Escriba aquí su código
import numpy as np
np.random.seed(54321)